In [1]:
cd langdata

/projects/metis2/anubha/research/Multilingual-Fig-QA/langdata


In [2]:
#Reading Dataset

In [3]:
ls

en_dev.csv  en_train.csv  hi.csv  id.csv  jv.csv  kn.csv  su.csv  sw.csv


In [4]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

### Experiment 6.2.3

In [5]:
import pandas as pd

In [6]:
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [170]:
#load other data
df = pd.read_csv("su.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,hirup kawas kurung batok,hirup kawas kakekang,hirup kawas bebas.,0
1,hirup kawas murag bulu bitis.,hirup kawas kakekang,hirup kawas bebas.,1
2,jadi jalma teh kudu hampang birit,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,0
3,jadi jalma teh kudu beurat birit.,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,1
4,Mamat lumpatna siga uncal,mamat lumpat siga gancang,mamat lumpat siga laun.,0


In [171]:
#Add id to the loaded data
count =0
c=[]
for i in range(int(df.shape[0]/2)):
    c.append(count)
    c.append(count)
    count+=1
df["id"] = c  

In [172]:
df.head()

,startphrase,ending1,ending2,labels,id
0,hirup kawas kurung batok,hirup kawas kakekang,hirup kawas bebas.,0,0
1,hirup kawas murag bulu bitis.,hirup kawas kakekang,hirup kawas bebas.,1,0
2,jadi jalma teh kudu hampang birit,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,0,1
3,jadi jalma teh kudu beurat birit.,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,1,1
4,Mamat lumpatna siga uncal,mamat lumpat siga gancang,mamat lumpat siga laun.,0,2


In [173]:
#df_original = english data
#df = language data
#k = number of datapoints to be added
#pair = in pair / not

def adding_datapoints(df_original, df, k, pair):
    df_final = pd.DataFrame(columns=df_original.columns)
    if pair:
        df_pair = df.sample(n=k, random_state=42)
        for index, r in df_pair.iterrows():
            r_id = r['id']
            w = df.loc[df['id'] == r_id]
            w.sort_values(by=['labels'], inplace=True)
            df=df.drop(w.index)
            w= w.drop(['id'], axis=1)
            df_final = df_final.append(w)
    else:
        df_pair = df.sample(n=k, random_state=42)
        df=df.drop(df_pair.index)
        w= df_pair.drop(['id'], axis=1)
        df_final = df_final.append(w)

    df_total = pd.concat([df_original,df_final])
    df_test = df.drop(['id'], axis=1)
    return df_total, df_test

    

In [191]:
num_samples = 6
lang = "su"

In [192]:
# #final dataframe, rows added in the end
# df_total = adding_datapoints(df_original, df, 10, True)
# df_total.tail()

#final dataframe, rows added in the end
df_total, df_test = adding_datapoints(df_original, df, num_samples, False)
df_total.tail()

/tmp/ipykernel_19845/2048821062.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)


,startphrase,ending1,ending2,labels
419,Dina éta balapan manehna jiga kuda hideung.,Dina éta balapan manehna jiga idola,Dina éta balapan manehna jiga jalmi nu teu dip...,1
565,Pernikahan mah ti jaman baheula gé asaan pasti...,Pernikahan mah aya nu awét,Pernikahan mah aya nu teu awét.,1
77,"lamun aya masalah, hidup teh karasa keur jalan...","lamun aya masalah, hidup teh karasa keurteu la...","lamun aya masalah, hidup teh karasa keur mulus.",1
181,Bumi na siga sarang na cucunguk.,Bumi na beresih pisan,Bumi na buladig.,1
284,Ngadangu carita anjeun mah kawas ngimpi labuh ...,Ngadangu carita anjeun mah ngareuwaskeun,Ngadangu carita anjeun mah nyenengkeun.,0


In [193]:
print(df.shape)
print(df_test.shape)

(600, 5)
(594, 4)


In [194]:
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand -> for paired data addition
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data -> for single data addition
winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand/'
no_winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data/'
folder_name = lang+ "/"
name = lang + "_" + str(num_samples)
df_total.to_csv(no_winogrand+'train/' +folder_name+ name+".csv", index = False)
df_test.to_csv(no_winogrand+'test/'+folder_name+ name+ ".csv", index = False)

### Removing Rows from testing data


### Experiment 6.2.2

In [33]:
import pandas as pd
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [35]:
#load other data
df = pd.read_csv("sw.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,Watoto ni malaika,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,0.0
1,Yeye ni mnyama.,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,1.0
2,Maneno yake ni kama kisu,Maneno yako yanaumiza,Maneno yake yanafariji,0.0
3,Maneno yake ni kama kinanda,Maneno yako yanaumiza,Maneno yake yanafariji,1.0
4,Tabasamu lake huangaza chumba kizima,Ana tabasamu zuri,Tabasamu lake linatisha,0.0


In [36]:
 df_total = pd.concat([df_original,df])

In [37]:
df_total.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0.0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1.0
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0.0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1.0
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0.0


In [38]:
print(df_original.shape)
print(df.shape)
print(df_total.shape)

(1458, 4)
(798, 4)
(2256, 4)


In [40]:
file = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/exp6.2.2/'
name = "eng_sw"
df_total.to_csv(file+name+ ".csv", index = False)